In [ ]:
import boto3

# ---- CONFIG ----
AWS_ACCESS_KEY_ID = "YOUR_AWS_ACCESS_KEY_ID"
AWS_SECRET_ACCESS_KEY = "YOUR_AWS_SECRET_ACCESS_KEY"
REGION = "us-west-2"  # Change as needed
STACK_IDENTIFIER = "trupti-sandbox-51db922282"  # Put your stack/environment substring here
# ----------------

# Create a session (you can also use environment variables or AWS profile)
session = boto3.Session(
    aws_access_key_id=AWS_ACCESS_KEY_ID,
    aws_secret_access_key=AWS_SECRET_ACCESS_KEY,
    region_name=REGION
)

dynamodb = session.client('dynamodb')

# List all tables and filter by stack/env if needed
def list_tables(filter_str=None):
    paginator = dynamodb.get_paginator('list_tables')
    print(f"🔍 Fetching DynamoDB tables in region: {REGION} ...")
    found = []
    for page in paginator.paginate():
        for table in page['TableNames']:
            if (not filter_str) or (filter_str in table):
                found.append(table)
    return found

if __name__ == "__main__":
    # Find all tables that contain your stack/env substring
    matching_tables = list_tables(STACK_IDENTIFIER)
    if not matching_tables:
        print(f"❌ No tables found with: {STACK_IDENTIFIER}")
    else:
        print("✅ Tables found:")
        for t in matching_tables:
            print(f"   - {t}")

        # Optionally, show more info about each table
        print("\nDetailed Table Info:")
        for t in matching_tables:
            desc = dynamodb.describe_table(TableName=t)['Table']
            print(f"Table: {t}")
            print(f"  - ARN: {desc['TableArn']}")
            print(f"  - Created: {desc['CreationDateTime']}")
            print(f"  - Item count: {desc.get('ItemCount', 'N/A')}")
            print(f"  - Status: {desc['TableStatus']}")
            print()
